In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple

def build_graph(distances_calculated: Dict[Tuple[str, str], float]) -> nx.Graph:
    G = nx.Graph()
    for (school1, school2), distance in distances_calculated.items():
        G.add_edge(school1, school2, weight=distance)
    return G

def draw_nodes_and_edges(G: nx.Graph, route_so_far: List[str], current_school: str, pos: Dict[str, Tuple[float, float]]):
    nx.draw_networkx_nodes(G, pos)
    nx.draw_networkx_nodes(G, pos, nodelist=route_so_far, node_color='y', label="Visited")
    nx.draw_networkx_nodes(G, pos, nodelist=[current_school], node_color='g', label="Current")
    nx.draw_networkx_edges(G, pos, alpha=0.2)
    for i in range(len(route_so_far) - 1):
        nx.draw_networkx_edges(G, pos, edgelist=[(route_so_far[i], route_so_far[i+1])], edge_color='r', width=2)

def visualize_route(distances_calculated: Dict[Tuple[str, str], float], route_so_far: List[str], current_school: str):
    G = build_graph(distances_calculated)
    pos = nx.spring_layout(G)
    plt.clf()
    draw_nodes_and_edges(G, route_so_far, current_school, pos)
    nx.draw_networkx_labels(G, pos)
    plt.legend()
    plt.draw()
    plt.pause(0.1)

def create_adjacent_schools(distances_calculated: Dict[Tuple[str, str], float]) -> Dict[str, List[Tuple[str, float]]]:
    adjacent_schools = {}
    sorted_distances = sorted(distances_calculated.items(), key=lambda item: item[1])
    schools = set()

    for pair in distances_calculated.keys():
        schools.add(pair[0])
        schools.add(pair[1])

    for school in schools:
        adjacent_schools[school] = []

    for pair, distance in sorted_distances:
        school1, school2 = pair
        adjacent_schools[school1].append((school2, distance))
        adjacent_schools[school2].append((school1, distance))

    return adjacent_schools

def find_optimal_route_with_visualization(distances_calculated: Dict[Tuple[str, str], float]) -> List[str]:
    adjacent_schools = create_adjacent_schools(distances_calculated)
    starting_pair = min(distances_calculated, key=distances_calculated.get)
    route = [starting_pair[0], starting_pair[1]]
    visited = {starting_pair[0], starting_pair[1]}
    current_school = starting_pair[1]

    plt.figure(figsize=(10, 6))

    total_schools = len(adjacent_schools)
    while len(visited) < total_schools:
        visualize_route(distances_calculated, route, current_school)
        found = False
        for next_school, _ in adjacent_schools[current_school]:
            if next_school not in visited:
                route.append(next_school)
                visited.add(next_school)
                current_school = next_school
                found = True
                break
        if not found:
            break

    plt.show(block=True)
    return route